In [155]:
#'.*id[1−5]?[0-9]{6}.htm'

In [156]:
import pandas as pd

In [157]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = './testapp-60fcd-f547300981c5.json'
VIEW_ID = '170130817'

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
  analytics = build('analyticsreporting', 'v4', credentials=credentials)
  return analytics

def get_report(analytics, pageTokenVar):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '700daysAgo', 'endDate': 'yesterday'}],
          'metrics': [{'expression': 'ga:pageviews'},{'expression': 'ga:entrances'}],
          'dimensions': [{'name': 'ga:pagePath'}],
          'dimensionFilterClauses': [{
                    'operator': "AND",
                    'filters': [
                          {
                          'dimensionName': 'ga:pagePath',
                          'operator': 'REGEXP',
                          'expressions': [
                            '.*id[0-9]{7}.htm'
                          ]},
                          {
                          'dimensionName': 'ga:pagePath',
                          'operator': 'REGEXP',
                          'expressions': [
                            '^[^?&]*$'
                          ]}
                    ]    
          }],
          'metricFilterClauses': [{ 
              "filters": [
                        {
                          'metricName': 'ga:pageviews',
                          'not': 'false',
                          'operator': 'LESS_THAN',
                          'comparisonValue': '6'
                        }
                    ]   
          }],
          'pageSize': 100000,
          'pageToken': pageTokenVar,
          'samplingLevel': 'LARGE'
          
        }],
        'useResourceQuotas': 'true'
      }
  ).execute()
    
def handle_report(analytics,pagetoken,rows):  

    response = get_report(analytics, pagetoken)

    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew
    print("len(rows): " + str(len(rows)))

    if pagetoken != None:
        return handle_report(analytics,pagetoken,rows)
    else:
        return rows

def main():    
    analytics = initialize_analyticsreporting()
    
    global dfanalytics
    dfanalytics = []

    rows = []
    rows = handle_report(analytics,'0',rows)

    dfanalytics = pd.DataFrame(list(rows))

if __name__ == '__main__':
  main()

len(rows): 100000
len(rows): 200000
len(rows): 222989


In [158]:
tags = dfanalytics['dimensions'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'pagePath')
dfanalytics = pd.concat([dfanalytics[:], tags[:]], axis=1)  
dfanalytics

,dimensions,metrics,pagePath
0,[/-id5378225.html],"[{'values': ['1', '0']}]",/-id5378225.html
1,[/-id7091834.html],"[{'values': ['3', '0']}]",/-id7091834.html
2,[/-id7252099.html],"[{'values': ['3', '0']}]",/-id7252099.html
3,[/-id7277672.html],"[{'values': ['3', '0']}]",/-id7277672.html
4,[/-id7393854.html],"[{'values': ['3', '0']}]",/-id7393854.html
5,[/-id7987322.html],"[{'values': ['1', '1']}]",/-id7987322.html
6,[/-id8943756.html],"[{'values': ['2', '2']}]",/-id8943756.html
7,[/-id8943976.html],"[{'values': ['2', '2']}]",/-id8943976.html
8,[/.../ein-callboy-packt-aus-id2153815.html],"[{'values': ['1', '0']}]",/.../ein-callboy-packt-aus-id2153815.html
9,[/«Bachelorette»-Muskelprotz-mit-Sex-Beichte-S...,"[{'values': ['2', '0']}]",/«Bachelorette»-Muskelprotz-mit-Sex-Beichte-Sa...


In [159]:
dfanalytics = dfanalytics.drop("dimensions",1)
dfanalytics = dfanalytics.drop("metrics",1)

In [160]:
dfanalytics

,pagePath
0,/-id5378225.html
1,/-id7091834.html
2,/-id7252099.html
3,/-id7277672.html
4,/-id7393854.html
5,/-id7987322.html
6,/-id8943756.html
7,/-id8943976.html
8,/.../ein-callboy-packt-aus-id2153815.html
9,/«Bachelorette»-Muskelprotz-mit-Sex-Beichte-Sa...


In [161]:
dfanalytics.to_csv('gaapi.csv', sep='\t')
